In [1]:
#imports
import pandas as pd
import pandas_ta as ta
import requests
import datetime
from backtesting import Backtest, Strategy
import backtrader as bt

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### My Fucntions

In [3]:
#Function to download data from Tiingo API
def get_tiingo_data(ticker, api, startDate='1995-01-01', endDate='2024-03-25'):
    #call Tiingo API and get ticker data - see Tiingo Docs
    headers = {'Content-Type': 'application/json'}
    r = requests.get(f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={startDate}&endDate={endDate}&token={api}", headers=headers)
    if r.status_code != 200: #print in case of any error
        print("Error number: ", r.status_code)
    #Create a Pandas DataFrame
    data = pd.DataFrame(r.json())
    # Format date into timestamp
    data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    #set date as index
    data.set_index('date', inplace=True)
    #Drop cols with non ajusted prices
    data.drop(columns=['close',
                      'high',
                      'low',
                      'open',
                      'divCash',
                      'splitFactor',
                      'adjVolume'], inplace=True)
    #Rename columns names
    data.rename(columns={'adjClose': 'Close',
                        'adjHigh': 'High',
                        'adjLow': 'Low',
                        'adjOpen': 'Open',
                        'volume': 'Volume'}, inplace=True)
    return data

In [7]:
def calculateIndicators(data):
    #Calculate indicators using pandas_ta
    data['ema20'] = ta.ema(data['Close'], length=20)
    data['ema25'] = ta.ema(data['Close'], length=25)
    data['ema35'] = ta.ema(data['Close'], length=35)
    data['ema60'] = ta.ema(data['Close'], length=60)
    data['coppock'] = ta.coppock(data['Close'], length=14)
    data['CMO'] = ta.cmo(data['Close'], length=14)
    data['rsx'] = ta.rsx(data['Close'], length=14) #0 to 100
    data['atr'] = ta.atr(high=data['High'], low=data['Low'], close=data['Close'], length=14)
    data['AO'] = ta.ao(high=data['High'], low=data['Low'])
    data['ChoppinessIndex'] = ta.chop(high=data['High'], low=data['Low'], close=data['C'])
    data = data.dropna()
    return data

#For information about pandas_ta indicators: https://github.com/twopirllc/pandas-ta/blob/main/README.md#volatility-14

### Code

In [8]:
#Load tiingo API from file
with open('tiingoAPI.txt', 'r') as file:
    tiingo_api = file.read().strip()

In [9]:
#Download assets data
ko = get_tiingo_data(ticker='ko', api=tiingo_api)
exon = get_tiingo_data(ticker='xom', api=tiingo_api)
jnj = get_tiingo_data(ticker='jnj', api=tiingo_api)
tesla = get_tiingo_data(ticker='tsla', api=tiingo_api)
ford = get_tiingo_data(ticker='f', api=tiingo_api)
apple = get_tiingo_data(ticker='aapl', api=tiingo_api)
coinbase = get_tiingo_data(ticker='coin', api=tiingo_api)
intel = get_tiingo_data(ticker='intc', api=tiingo_api)
dow = get_tiingo_data(ticker='dow', api=tiingo_api)
jp_morgan = get_tiingo_data(ticker='jpm', api=tiingo_api)
hsbc = get_tiingo_data(ticker='hsbc', api=tiingo_api)
hsbc

,Volume,Close,High,Low,Open
date,,,,,
1999-07-16,231400,18.519110,18.593711,18.387811,18.537014
1999-07-19,158200,18.557903,19.005512,18.352002,18.984624
1999-07-20,79400,18.166990,18.426604,18.128197,18.426604
1999-07-21,75200,18.202799,18.259496,18.017787,18.166990
1999-07-22,33500,18.166990,18.202799,18.089404,18.184894
...,...,...,...,...,...
2024-03-19,1293965,38.500000,38.570000,38.340000,38.370000
2024-03-20,2356977,39.060000,39.060000,38.345000,38.380000
2024-03-21,2476832,39.500000,39.735000,39.380000,39.410000


In [10]:
#calculate assets indicators
ko = calculateIndicators(ko)
exon = calculateIndicators(exon)
jnj = calculateIndicators(jnj)
tesla = calculateIndicators(tesla)
ford = calculateIndicators(ford)
apple = calculateIndicators(apple)
coinbase = calculateIndicators(coinbase)
intel = calculateIndicators(intel)
dow = calculateIndicators(dow)
jp_morgan = calculateIndicators(jp_morgan)
hsbc = calculateIndicators(hsbc)
hsbc

,Volume,Close,High,Low,Open,ema20,ema25,ema35,ema60,coppock,CMO,rsx,atr,AO
date,,,,,,,,,,,,,,
1999-10-08,32500,17.743988,17.783211,17.535803,17.650455,17.798455,17.851702,17.907648,18.111538,-5.132351,-3.646082,48.364729,0.405517,-0.432165
1999-10-11,34200,17.535803,17.614249,17.406064,17.575026,17.773441,17.827402,17.886990,18.092661,-4.041441,-8.830807,48.977787,0.400632,-0.424081
1999-10-12,94700,17.234086,17.499597,16.914265,17.499597,17.722074,17.781762,17.850717,18.064511,-4.170830,-15.894268,48.241402,0.416584,-0.454608
1999-10-13,159300,16.669875,16.914265,16.500913,16.914265,17.621864,17.696232,17.785115,18.018785,-4.485128,-27.245785,45.264967,0.439429,-0.616417
1999-10-14,81900,17.083227,17.083227,16.630651,16.763407,17.570565,17.649078,17.746121,17.988111,-4.103312,-14.999920,43.166868,0.440377,-0.735418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-19,1293965,38.500000,38.570000,38.340000,38.370000,37.748541,37.710808,37.671418,37.592024,4.117966,15.712319,61.091726,0.564401,0.529299
2024-03-20,2356977,39.060000,39.060000,38.345000,38.380000,37.873442,37.814592,37.748562,37.640154,4.895000,24.069036,64.059562,0.575158,0.654530
2024-03-21,2476832,39.500000,39.735000,39.380000,39.410000,38.028352,37.944239,37.845864,37.701133,5.882012,29.946011,67.813623,0.582289,0.995149


In [8]:
#export to csv to serve as a backup
ko.to_csv("KO.csv", index=True)
exon.to_csv("EXON.csv", index=True)
jnj.to_csv("JNJ.csv", index=True)
tesla.to_csv("TESLA.csv", index=True)
ford.to_csv("FORD.csv", index=True)
apple.to_csv("APPLE.csv", index=True)
coinbase.to_csv("COINBASE.csv", index=True)
intel.to_csv("INTEL.csv", index=True)
dow.to_csv("DOW.csv", index=True)
jp_morgan.to_csv("JPM.csv", index=True)
hsbc.to_csv("HSBC.csv", index=True)

In [9]:
"""#Import data from cvs
ko = pd.read_csv("KO.csv")
exon = pd.read_csv("EXON.csv")
jnj = pd.read_csv("JNJ.csv")
tesla = pd.read_csv("TESLA.csv")
ford = pd.read_csv("FORD.csv")
apple = pd.read_csv("APPLE.csv")
coinbase = pd.read_csv("COINBASE.csv")
intel = pd.read_csv("INTEL.csv")
dow = pd.read_csv("DOW.csv")
jp_morgan = pd.read_csv("JPM.csv")
hsbc = pd.read_csv("HSBC.csv")"""

'#Import data from cvs\nko = pd.read_csv("KO.csv")\nexon = pd.read_csv("EXON.csv")\njnj = pd.read_csv("JNJ.csv")\ntesla = pd.read_csv("TESLA.csv")\nford = pd.read_csv("FORD.csv")\napple = pd.read_csv("APPLE.csv")\ncoinbase = pd.read_csv("COINBASE.csv")\nintel = pd.read_csv("INTEL.csv")\ndow = pd.read_csv("DOW.csv")\njp_morgan = pd.read_csv("JPM.csv")\nhsbc = pd.read_csv("HSBC.csv")'

### Backtest with backtesting.py

In [76]:
class trading_strategy(Strategy):
    
    def init(self):
        self.already_bought = False
        self.stop_loss = 0
        self.take_profit = 0        
        
        
    def next(self):
        if self.data.ema35[-2] < self.data.Close[-2] and self.data.ema35[-1] > self.data.Close[-1] and self.already_bought == False:
            self.stop_loss = self.data.Close[-1] - 3 * self.data.atr[-1]
            self.take_profit = self.data.Close[-1] + 5 * self.data.atr[-1]
            self.buy(tp=self.take_profit, sl=self.stop_loss)
            self.already_bought = True
        #elif self.data.Close[-1] < self.stop_loss or self.data.Close[-1] > self.take_profit and self.already_bought == True:
        elif self.position:
            self.already_bought = False
        else:
            pass

In [77]:
#Run the startegy
#bt = Backtest(exon, trading_strategy, cash=1000, commission=.002, exclusive_orders=True)
bt_exon = Backtest(exon, trading_strategy, cash=1000, commission=.002)
results_exon = bt_exon.run()
results_exon

Start                     1995-03-28 00:00:00
End                       2024-03-25 00:00:00
Duration                  10590 days 00:00:00
Exposure Time [%]                   57.322921
Equity Final [$]                  1838.895718
Equity Peak [$]                   2483.390499
Return [%]                          83.889572
Buy & Hold Return [%]             1613.337598
Return (Ann.) [%]                    2.125432
Volatility (Ann.) [%]               18.166776
Sharpe Ratio                         0.116996
Sortino Ratio                        0.172475
Calmar Ratio                         0.047485
Max. Drawdown [%]                  -44.760092
Avg. Drawdown [%]                   -5.227641
Max. Drawdown Duration     3563 days 00:00:00
Avg. Drawdown Duration      141 days 00:00:00
# Trades                                   59
Win Rate [%]                        42.372881
Best Trade [%]                       48.32392
Worst Trade [%]                    -11.380625
Avg. Trade [%]                    

In [78]:
bt_exon.plot()
#bt.plot(resample=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fi

GridPlot(id='p3309', ...)

In [79]:
#Run the strategy for all assets
bt_ko = Backtest(ko, trading_strategy, cash=1000, commission=.002)
results_ko = bt_ko.run()
bt_jnj = Backtest(jnj, trading_strategy, cash=1000, commission=.002)
results_jnj = bt_jnj.run()
bt_tesla = Backtest(tesla, trading_strategy, cash=1000, commission=.002)
results_tesla = bt_tesla.run()
bt_ford = Backtest(ford, trading_strategy, cash=1000, commission=.002)
results_ford = bt_ford.run()
bt_apple = Backtest(apple, trading_strategy, cash=1000, commission=.002)
results_apple = bt_apple.run()
bt_coinbase = Backtest(coinbase, trading_strategy, cash=1000, commission=.002)
results_coinbase = bt_coinbase.run()
bt_intel = Backtest(intel, trading_strategy, cash=1000, commission=.002)
results_intel = bt_intel.run()
bt_dow = Backtest(dow, trading_strategy, cash=1000, commission=.002)
results_dow = bt_dow.run()
bt_jp_morgan = Backtest(jp_morgan, trading_strategy, cash=1000, commission=.002)
results_jp_morgan = bt_jp_morgan.run()
bt_hsbc = Backtest(hsbc, trading_strategy, cash=1000, commission=.002)
results_hsbc = bt_hsbc.run()

In [80]:
#Create a dataframe with results to better compare
exon_data = results_exon.to_frame(name='exon')
ko_data = results_ko.to_frame(name='ko')
jnj_data = results_jnj.to_frame(name='jnj')
tesla_data = results_tesla.to_frame(name='tesla')
ford_data = results_ford.to_frame(name='ford')
apple_data = results_apple.to_frame(name='apple')
coinbase_data = results_coinbase.to_frame(name='coin')
intel_data = results_intel.to_frame(name='intel')
dow_data = results_dow.to_frame(name='dow')
jpm_data = results_jp_morgan.to_frame(name='jpm')
hsbc_data = results_hsbc.to_frame(name='hsbc')

In [81]:
#concatenate dfs
all_results = pd.concat([
    exon_data,
    ko_data,
    jnj_data,
    tesla_data,
    ford_data,
    apple_data,
    coinbase_data,
    intel_data,
    dow_data,
    jpm_data,
    hsbc_data
], axis=1)

all_results

,exon,ko,jnj,tesla,ford,apple,coin,intel,dow,jpm,hsbc
Start,1995-03-28 00:00:00,1995-03-28 00:00:00,1995-03-28 00:00:00,2010-09-22 00:00:00,1995-03-28 00:00:00,1995-03-28 00:00:00,2021-07-08 00:00:00,1995-03-28 00:00:00,2019-06-13 00:00:00,1995-03-28 00:00:00,1999-10-08 00:00:00
End,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00
Duration,10590 days 00:00:00,10590 days 00:00:00,10590 days 00:00:00,4933 days 00:00:00,10590 days 00:00:00,10590 days 00:00:00,991 days 00:00:00,10590 days 00:00:00,1747 days 00:00:00,10590 days 00:00:00,8935 days 00:00:00
Exposure Time [%],57.322921,60.419236,52.103028,51.927037,34.991095,33.278531,53.587116,35.922729,46.760797,52.637348,51.689958
Equity Final [$],1838.895718,1989.991275,7473.477771,509.707942,767.093271,8246.871996,2727.715721,634.660344,2377.481445,543.894168,949.700288
Equity Peak [$],2483.390499,2164.460701,8701.183359,1435.689495,1554.532854,9445.446798,2727.715721,1577.317487,2793.37554,1884.874716,1633.60399
Return [%],83.889572,98.999127,647.347777,-49.029206,-23.290673,724.6872,172.771572,-36.533966,137.748145,-45.610583,-5.029971
Buy & Hold Return [%],1613.337598,767.645477,1900.579015,12931.957725,138.530752,66216.87435,14.499161,1228.979272,39.49981,3641.60248,122.723328
Return (Ann.) [%],2.125432,2.404235,7.191072,-4.87362,-0.91125,7.556123,44.808029,-1.557488,19.873226,-2.080641,-0.21111
Volatility (Ann.) [%],18.166776,16.397172,15.480047,32.948676,23.137591,29.984228,89.124509,22.982334,26.90179,23.445974,16.017873


In [83]:
#Print some important results
positives, negatives = sum(1 for c in all_results.iloc[4] if c > 1000), sum(1 for c in all_results.iloc[4] if c <= 1000)
print(f"Number of assets with POSITIVE returns: {positives}")
print(f"Number of assets with NEGATIVE returns: {negatives}")
print(all_results.iloc[4])
print(f"End portfolio value of {all_results.iloc[4].sum():.3f}$ out of 11000$")

Number of assets with POSITIVE returns: 6
Number of assets with NEGATIVE returns: 5
exon     1838.895718
ko       1989.991275
jnj      7473.477771
tesla     509.707942
ford      767.093271
apple    8246.871996
coin     2727.715721
intel     634.660344
dow      2377.481445
jpm       543.894168
hsbc      949.700288
Name: Equity Final [$], dtype: object
End portfolio value of 28059.490$ out of 11000$
exon     42.372881
ko       46.601942
jnj      52.873563
tesla         38.0
ford     40.350877
apple    55.072464
coin     55.555556
intel    42.105263
dow      47.619048
jpm      43.925234
hsbc          41.0
Name: Win Rate [%], dtype: object
